# BUILD THE NEURAL NETWORK
Mạng nơ-ron bao gồm các lớp / mô-đun thực hiện các hoạt động trên dữ liệu. Không gian tên `torch.nn` cung cấp tất cả các khối xây dựng mà bạn cần để xây dựng mạng nơ-ron của riêng mình. Mọi mô-đun trong PyTorch phân lớp `nn.Module`. Mạng nơ-ron là một mô-đun chính nó bao gồm các mô-đun (lớp) khác. Cấu trúc lồng nhau này cho phép xây dựng và quản lý các kiến trúc phức tạp một cách dễ dàng.

Trong các phần sau, chúng tôi sẽ xây dựng mạng nơ-ron để phân loại hình ảnh trong tập dữ liệu FashionMNIST.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Nhận thiết bị để đào tạo
Chúng tôi muốn có thể đào tạo mô hình của mình trên một bộ tăng tốc phần cứng như GPU, nếu nó có sẵn. Hãy kiểm tra xem liệu `torch.cuda` có khả dụng hay không, nếu không, chúng ta sẽ tiếp tục sử dụng CPU.

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


## Xác định Lớp
Chúng tôi xác định mạng nơ-ron của mình bằng cách phân lớp `nn.Module` và khởi tạo các lớp mạng nơ-ron trong `__init__`. Mọi lớp con `nn.Module` thực hiện các thao tác trên dữ liệu đầu vào trong phương thức `forward `.

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Chúng tôi tạo một phiên bản của `NeuralNetwork`, và di chuyển nó vào `device` và in cấu trúc của nó.

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


Để sử dụng mô hình, chúng tôi chuyển nó dữ liệu đầu vào. Điều này thực hiện `forward` của mô hình, cùng với một số hoạt động nền ([background operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866)). Không gọi trực tiếp `model.forward()`!

Gọi mô hình trên đầu vào trả về một tensor 10 chiều với các giá trị dự đoán thô cho mỗi lớp. Chúng tôi nhận được các xác suất dự đoán bằng cách chuyển nó qua một phiên bản của mô-đun `nn.Softmax`.

In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2], device='cuda:0')


## Lớp mô hình
Hãy chia nhỏ các lớp trong mô hình FashionMNIST. Để minh họa, chúng tôi sẽ lấy một minibatch mẫu gồm 3 hình ảnh có kích thước 28x28 và xem điều gì sẽ xảy ra với nó khi chúng tôi chuyển nó qua mạng.

In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


## `nn.Flatten`
Chúng tôi khởi tạo lớp `nn.Flatten` để chuyển đổi mỗi hình ảnh 2D 28x28 thành một mảng liền kề gồm các giá trị 784 pixel (kích thước của minibatch (ở dim = 0) được duy trì).

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


## `nn.Linear`
Lớp tuyến tính là một mô-đun áp dụng một phép biến đổi tuyến tính trên đầu vào bằng cách sử dụng trọng số và độ lệch được lưu trữ của nó.

In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


## `nn.ReLU`
Kích hoạt phi tuyến tính là những gì tạo ra ánh xạ phức tạp giữa đầu vào và đầu ra của mô hình. Chúng được áp dụng sau các phép biến đổi tuyến tính để giới thiệu tính phi tuyến, giúp mạng nơ-ron học được nhiều loại hiện tượng.

Trong mô hình này, chúng tôi sử dụng `nn.ReLU` giữa các lớp tuyến tính của chúng tôi, nhưng có các kích hoạt khác để giới thiệu tính phi tuyến tính trong mô hình của bạn.

In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1236,  0.0642, -0.9711, -0.3984, -0.1394, -0.1302,  0.3699, -0.8841,
         -0.0602,  0.4747,  0.5753, -0.1412, -0.1086,  0.4065,  0.0610, -0.1744,
          0.2672,  0.7211,  0.0222, -0.0526],
        [ 0.1015,  0.0934, -0.5248, -0.1563, -0.3430, -0.3936,  0.3076, -0.6872,
         -0.0267,  0.4958,  0.5227,  0.0645, -0.1731,  0.4286, -0.3167, -0.0163,
          0.2338,  0.8249, -0.1315,  0.1254],
        [ 0.1871,  0.4144, -0.5178,  0.0674,  0.2161, -0.4167,  0.3593, -0.8000,
          0.2020,  0.5743,  0.2168,  0.0552, -0.2043,  0.0948, -0.0711,  0.0562,
          0.1454,  0.9215,  0.0964, -0.0305]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.1236, 0.0642, 0.0000, 0.0000, 0.0000, 0.0000, 0.3699, 0.0000, 0.0000,
         0.4747, 0.5753, 0.0000, 0.0000, 0.4065, 0.0610, 0.0000, 0.2672, 0.7211,
         0.0222, 0.0000],
        [0.1015, 0.0934, 0.0000, 0.0000, 0.0000, 0.0000, 0.3076, 0.0000, 0.0000,
         0.4958, 0.5227, 0.0645, 0.0000, 0.4286, 0.000

## `nn.Sequential`
`nn.Sequential` là một vùng chứa các mô-đun có thứ tự. Dữ liệu được chuyển qua tất cả các mô-đun theo thứ tự như đã xác định. Bạn có thể sử dụng các vùng chứa tuần tự để kết hợp một mạng nhanh chóng như `seq_modules`.

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

## `nn.Softmax`
Lớp tuyến tính cuối cùng của mạng nơ-ron trả về logits - giá trị thô trong [-infty, infty] - được chuyển đến mô-đun `nn.Softmax`. `logits` được chia tỷ lệ thành các giá trị [0, 1] đại diện cho các xác suất dự đoán của mô hình cho mỗi lớp. tham số `dim` cho biết thứ nguyên mà các giá trị phải cộng lại bằng 1. 

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## Thông số mô hình
Nhiều lớp bên trong mạng nơ-ron được tham số hóa, tức là có trọng số và độ lệch liên quan được tối ưu hóa trong quá trình đào tạo. Phân lớp `nn.Module` tự động theo dõi tất cả các trường được xác định bên trong đối tượng mô hình của bạn và làm cho tất cả các tham số có thể truy cập được bằng cách sử dụng phương thức `parameters()` hoặc `names_parameters()` của mô hình.

Trong ví dụ này, chúng tôi lặp lại từng tham số và in kích thước của nó và bản xem trước các giá trị của nó.

In [12]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0013, -0.0036, -0.0162,  ..., -0.0345,  0.0179, -0.0050],
        [-0.0029,  0.0032, -0.0163,  ..., -0.0336,  0.0169,  0.0209]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0126, -0.0333], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0023, -0.0124, -0.0187,  ...,  0.0247, -0.0285, -0.0139],
        [-0.0259,  0.0382,  0.0435,  ..., -0.0129,  0.0384, -0.0344]],
       device='cuda

## Further Reading
- [`torch.nn` API](https://pytorch.org/docs/stable/nn.html)